In [ ]:
import bcrypt
import sqlite3
import time
import json
import random

# Create users table
def create_db():
    conn = sqlite3.connect('crypto_trading.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY,
            username TEXT UNIQUE,
            password TEXT,
            balance REAL DEFAULT 10000.0
        )
    ''')
    conn.commit()
    conn.close()

# Hash password
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

# Verify password
def verify_password(stored_password, provided_password):
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_password)

# User registration
def register_user(username, password):
    conn = sqlite3.connect('crypto_trading.db')
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hash_password(password)))
        conn.commit()
        print("Registration successful.")
    except sqlite3.IntegrityError:
        print("Username already exists.")
    finally:
        conn.close()

# User login
def login_user(username, password):
    conn = sqlite3.connect('crypto_trading.db')
    c = conn.cursor()
    c.execute("SELECT password FROM users WHERE username = ?", (username,))
    user = c.fetchone()
    if user and verify_password(user[0], password):
        print("Login successful.")
        return True
    else:
        print("Invalid username or password.")
        return False

# Initialize blockchain
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(previous_hash='1')  # Genesis block
        self.populate_chain()

    def create_block(self, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'transactions': [],
            'previous_hash': previous_hash
        }
        self.chain.append(block)
        return block

    def add_transaction(self, transaction):
        self.chain[-1]['transactions'].append(transaction)

    def populate_chain(self):
        # Generate 500 dummy transactions for 500 users
        for i in range(500):
            username = f'User{i + 1}'
            amount = random.uniform(1, 1000)
            transaction = {
                'username': username,
                'amount': amount,
                'timestamp': time.time()
            }
            self.add_transaction(transaction)

    def view_blockchain(self):
        return json.dumps(self.chain, indent=4)

# Initialize blockchain
blockchain = Blockchain()

# Function to buy cryptocurrency
def buy_crypto(username, cryptocurrency, amount):
    conn = sqlite3.connect('crypto_trading.db')
    c = conn.cursor()
    c.execute("SELECT balance FROM users WHERE username = ?", (username,))
    user = c.fetchone()

    if user:
        current_balance = user[0]
        total_cost = amount * get_crypto_price(cryptocurrency)  # Assuming get_crypto_price is a function that returns price

        if current_balance >= total_cost:
            # Relaxed insider trading check
            # Here we can allow the transaction without strict checks
            transaction = {
                'username': username,
                'cryptocurrency': cryptocurrency,
                'amount': amount,
                'timestamp': time.time()
            }
            blockchain.add_transaction(transaction)
            new_balance = current_balance - total_cost
            c.execute("UPDATE users SET balance = ? WHERE username = ?", (new_balance, username))
            conn.commit()
            print(f"Successfully purchased {amount} of {cryptocurrency}. New balance: {new_balance:.2f}")
        else:
            print("Insufficient balance.")
    else:
        print("User not found.")

    conn.close()

# Function to simulate fetching cryptocurrency price
def get_crypto_price(cryptocurrency):
    # Placeholder function to simulate crypto price fetching
    return random.uniform(10, 100)  # Random price between 10 and 100

# Function to view blockchain history
def view_blockchain_history():
    print(blockchain.view_blockchain())

# Main program
def main():
    create_db()

    while True:
        print("1. Register")
        print("2. Login")
        print("3. View Blockchain History")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            username = input("Enter username: ")
            password = input("Enter password: ")
            register_user(username, password)
        elif choice == '2':
            username = input("Enter username: ")
            password = input("Enter password: ")
            if login_user(username, password):
                while True:
                    print("1. Buy Cryptocurrency")
                    print("2. View Blockchain History")
                    print("3. Logout")
                    sub_choice = input("Enter choice: ")
                    if sub_choice == '1':
                        cryptocurrency = input("Enter cryptocurrency: ")
                        amount = float(input("Enter amount: "))
                        buy_crypto(username, cryptocurrency, amount)
                    elif sub_choice == '2':
                        view_blockchain_history()
                    elif sub_choice == '3':
                        break
                    else:
                        print("Invalid choice.")
        elif choice == '3':
            view_blockchain_history()
        elif choice == '4':
            print("Exiting...")
            break
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    main()


1. Register
2. Login
3. View Blockchain History
4. Exit
[
    {
        "index": 1,
        "timestamp": 1727504444.9715133,
        "transactions": [
            {
                "username": "User1",
                "amount": 115.22373815100468,
                "timestamp": 1727504444.9715323
            },
            {
                "username": "User2",
                "amount": 838.3415776109858,
                "timestamp": 1727504444.971538
            },
            {
                "username": "User3",
                "amount": 347.83272414042096,
                "timestamp": 1727504444.9715397
            },
            {
                "username": "User4",
                "amount": 103.64920804788288,
                "timestamp": 1727504444.971541
            },
            {
                "username": "User5",
                "amount": 290.829804030839,
                "timestamp": 1727504444.9715421
            },
            {
                "username": "User6",
   